In [21]:
%%time
# Import Modules
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
pd.options.display.float_format = '{:,.2f}'.format
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score , confusion_matrix , precision_score , precision_recall_curve , f1_score
from sklearn.metrics import classification_report , roc_auc_score, recall_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
import sklearn.metrics as metrics
from joblib import dump, load
import joblib

Wall time: 0 ns


In [22]:
rfc = joblib.load('modelmmww.joblib')

In [23]:
df = pd.read_csv('RosterLinkedIn_YaleM2.csv')
#df2 = pd.read_csv('RosterLinkedIn_CornellW.csv')

In [24]:
#df['SameTennis_yn'] = (df['SameName_yn'] + df['tennis_yn'])
#df2['SameTennis_yn'] = (df2['SameName_yn'] + df2['tennis_yn'])


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 424 entries, 0 to 423
Data columns (total 33 columns):
FirstLast         424 non-null object
Criteria          424 non-null object
Year              424 non-null int64
YearsPlayed       424 non-null int64
RosterLink        40 non-null object
Hometown          40 non-null object
HS                37 non-null object
ClassYear         40 non-null object
College           424 non-null object
First             424 non-null object
Gender            424 non-null int64
Last              424 non-null object
MidInit           271 non-null object
MidName           35 non-null object
Name              40 non-null object
RosterYear        40 non-null object
LName             362 non-null object
SearchResult      362 non-null object
Headline          362 non-null object
Location          362 non-null object
Company           350 non-null object
School            362 non-null object
Activites         362 non-null object
fuzz_ratio        362 non-null 

In [26]:
#This was for CornellW_merge which had names with no result, e.g. Nan rows
df.dropna(subset = ['SearchResult'], inplace=True)
df.set_index('FirstLast', inplace=True, drop=False)
df = df.select_dtypes(exclude=['object'])
#df_label = df[['EC_Review']]
#df.drop(['EC_Review', 'prediction', 'Decade', 'YearsPlayed'], axis=1, inplace=True)

In [27]:
#Do not drop duplicates at this stage, because smae LinkedIn returned for several FirstLast names
#don't want to drop at index level
#df.drop_duplicates(inplace=True)

In [28]:
#df.drop(['MidInit', 'MidName'], axis=1, inplace=True)

In [30]:
df

Year  YearsPlayed  Gender  fuzz_ratio  fuzz_pratio  \
FirstLast                                                             
Victor Kiam      1947            1       1       22.00        27.00   
Charles Ritchie  1947            2       1       71.00        73.00   
James Boyle      1947            1       1      100.00       100.00   
Henry Ewald      1947            1       1       46.00        55.00   
Robert Sutton    1947            1       1       87.00        70.00   
...               ...          ...     ...         ...          ...   
Daniel Gale      2020            1       1       67.00        73.00   
Ryan Cheng       2020            4       1       63.00        67.00   
Dylan King       2020            4       1       78.00        88.00   
Cody Lin         2020            3       1       80.00        86.00   
Alan Sou         2020            3       1      100.00       100.00   

                 fuzz_tokensort  fuzz_tokenset  fuzz_wratio  tennis_yn  \
FirstLast                                                                
Victor Kiam               22.00          22.00        22.00       0.00   
Charles Ritchie           45.00          77.00        77.00       0.00   
James Boyle              100.00         100.00       100.00       0.00   
Henry Ewald               46.00          46.00        54.00       0.00   
Robert Sutton             87.00          87.00        87.00       0.00   
...                         ...            ...          ...        ...   
Daniel Gale               67.00          71.00        67.00       0.00   
Ryan Cheng                63.00          63.00        63.00       0.00   
Dylan King                82.00          83.00        82.00       1.00   
Cody Lin                  86.00          86.00        86.00       1.00   
Alan Sou                 100.00         100.00       100.00       0.00   

                 School_yn  SameName_yn  SameTennis_yn  Decade  
FirstLast                                                       
Victor Kiam           0.00         0.00           0.00    1940  
Charles Ritchie       1.00         0.00           0.00    1940  
James Boyle           1.00         1.00           1.00    1940  
Henry Ewald           1.00         0.00           0.00    1940  
Robert Sutton         0.00         0.00           0.00    1940  
...                    ...          ...            ...     ...  
Daniel Gale           1.00         0.00           0.00    2020  
Ryan Cheng            0.00         0.00           0.00    2020  
Dylan King            1.00         0.00           1.00    2020  
Cody Lin              1.00         0.00           1.00    2020  
Alan Sou              1.00         1.00           1.00    2020  

[362 rows x 13 columns]

In [31]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362 entries, Victor Kiam to Alan Sou
Data columns (total 13 columns):
Year              362 non-null int64
YearsPlayed       362 non-null int64
Gender            362 non-null int64
fuzz_ratio        362 non-null float64
fuzz_pratio       362 non-null float64
fuzz_tokensort    362 non-null float64
fuzz_tokenset     362 non-null float64
fuzz_wratio       362 non-null float64
tennis_yn         362 non-null float64
School_yn         362 non-null float64
SameName_yn       362 non-null float64
SameTennis_yn     362 non-null float64
Decade            362 non-null int64
dtypes: float64(9), int64(4)
memory usage: 39.6+ KB


In [32]:
X = df.values

In [33]:
X.shape

(362, 13)

In [34]:
rfc

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('rfc',
                                        RandomForestClassifier(bootstrap=True,
                                                               class_weight=None,
                                                               criterion='gini',
                                                               max_depth=None,
                                                               max_features='auto',
                                                               max_leaf_nodes=None,
                                                               min_impurity_decrease=0.0,
                  

In [35]:
rfcpredictX = rfc.predict(X)

In [36]:
rfcpredictX

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
       1., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0.,
       0., 0., 0., 1., 1., 1., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 1.,
       0., 1., 0., 0., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 1., 0.,
       0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       0., 0., 1., 0., 1.

In [37]:
df['prediction'] = rfcpredictX

In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 362 entries, Victor Kiam to Alan Sou
Data columns (total 14 columns):
Year              362 non-null int64
YearsPlayed       362 non-null int64
Gender            362 non-null int64
fuzz_ratio        362 non-null float64
fuzz_pratio       362 non-null float64
fuzz_tokensort    362 non-null float64
fuzz_tokenset     362 non-null float64
fuzz_wratio       362 non-null float64
tennis_yn         362 non-null float64
School_yn         362 non-null float64
SameName_yn       362 non-null float64
SameTennis_yn     362 non-null float64
Decade            362 non-null int64
prediction        362 non-null float64
dtypes: float64(10), int64(4)
memory usage: 42.4+ KB


In [39]:
df.to_csv('Predict_YaleM2_mmww.csv', index=True)